In [1]:
import sys
sys.path.append('../../../')

import pandas as pd
import numpy as np

import keras

from utils.processing.v02.dataset import TradingDataset
from utils.strategy import TradeStrategy
from utils.backtester import Backtester
import warnings

warnings.filterwarnings('ignore')

2025-03-13 20:06:01.189126: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
SEQUENCE_LENGTH = 9
SHIFT_SIZE = 3

# Loading the model

In [3]:
model = keras.models.load_model('../../../models/test/best_model_EURUSD_candle.keras')
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 512)            │     1,132,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │         4,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,410,027 (13.01 MB)

 Trainable params: 1,136,675 (4.34 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,273,352 (8.67 MB)

# Loading the previous sequences

In [32]:
base_df = pd.read_csv('../../../market_data.csv')


base_df['DATETIME'] = pd.to_datetime(base_df['DATETIME'])
base_df = base_df.set_index('DATETIME')
base_df.sort_index(inplace=True)

df = base_df.loc['2024-05-01 00:00:00':]
future_df = df[(-24 * 7):]

df = df[:(-24 * 7)]
print(df.shape)
df.tail()

(5198, 13)


,OPEN,HIGH,LOW,CLOSE,VOLUME,EMA,RSI,MACD,SIGNAL_LIGNE,UPPER_BAND,LOWER_BAND,BOLLINGER_PERCENT_B,ATR
DATETIME,,,,,,,,,,,,,
2025-03-03 22:00:00,1.04729,1.04852,1.04665,1.04816,5024,1.04648,66.40,0.00211,0.00186,1.04481,1.05139,0.49078,0.00193
2025-03-03 23:00:00,1.04814,1.04919,1.04795,1.04851,1527,1.04675,67.47,0.00209,0.00197,1.04513,1.05177,0.49138,0.00196
2025-03-04 00:00:00,1.04854,1.04883,1.04814,1.04883,723,1.04703,68.45,0.00207,0.00205,1.04548,1.05213,0.49632,0.00188
2025-03-04 01:00:00,1.04874,1.04924,1.04767,1.04852,1456,1.04723,66.35,0.00201,0.00209,1.04582,1.05236,0.58724,0.00179
2025-03-04 02:00:00,1.04853,1.04886,1.04769,1.04880,2205,1.04744,67.33,0.00196,0.00210,1.04617,1.05256,0.58849,0.00175


# Generate the next close price 

In [5]:
dataset = TradingDataset(df, shift_size=SHIFT_SIZE, sequence_length=SEQUENCE_LENGTH)
dataset.prepare_data()

In [6]:
last_prev_lines:np.ndarray = dataset.feature_sequence[-1]
prev_seq = last_prev_lines.reshape(1, dataset.SEQUENCE_LENGTH, last_prev_lines.shape[1])
predictions = model.predict(prev_seq)
predictions = dataset.target_scaler.inverse_transform(predictions)
print("========[ PREV CANDLE ]========\n"
      "HIGH: {}\n"
      "LOW: {}\n"
      "CLOSE: {}\n"
      "==============================".format(predictions[0][0], predictions[0][1], predictions[0][2]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
========[ PREV CANDLE ]========
HIGH: 1.0925246477127075
LOW: 1.091984748840332
CLOSE: 1.0918747186660767


In [35]:
dataset_backtest = TradingDataset(df, shift_size=SHIFT_SIZE, sequence_length=SEQUENCE_LENGTH)
dataset_backtest.prepare_data()

backtest_df = pd.DataFrame({
    'HIGH': [],
    'LOW': [],
    'CLOSE': [],
    'PREDICTED_HIGH': [],
    'PREDICTED_LOW': [],
    'PREDICTED_CLOSE': [],
    'DATETIME': []
}).set_index('DATETIME')

for i in range(0, len(future_df)):
    last_prev_lines = dataset_backtest.feature_sequence[-1]
    prev_seq = last_prev_lines.reshape(1, dataset_backtest.SEQUENCE_LENGTH, last_prev_lines.shape[1])
    predictions = model.predict(prev_seq)
    predictions = dataset.target_scaler.inverse_transform(predictions)

    next_row = pd.DataFrame({
        'HIGH': [future_df.iloc[i]['HIGH']],
        'LOW': [future_df.iloc[i]['LOW']],
        'CLOSE': [future_df.iloc[i]['CLOSE']],
        'PREDICTED_HIGH': [predictions[0][0]],
        'PREDICTED_LOW': [predictions[0][1]],
        'PREDICTED_CLOSE': [predictions[0][2]]
    }, index=[dataset_backtest.df.index[-1] + pd.Timedelta(hours=1)])

    backtest_df = pd.concat([backtest_df, next_row])
    updated_df = pd.concat([dataset_backtest.df, future_df.iloc[[i]]])
    dataset_backtest.update_df(updated_df)
	

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━

In [37]:
backtest_df.tail()

,HIGH,LOW,CLOSE,PREDICTED_HIGH,PREDICTED_LOW,PREDICTED_CLOSE
2025-03-12 22:00:00,1.08930,1.08850,1.08869,1.091771,1.090794,1.091148
2025-03-12 23:00:00,1.08892,1.08835,1.08841,1.090095,1.089924,1.090019
2025-03-13 00:00:00,1.08897,1.08866,1.08886,1.089889,1.089584,1.089689
2025-03-13 01:00:00,1.08900,1.08832,1.08856,1.089893,1.089552,1.089514
2025-03-13 02:00:00,1.08868,1.08826,1.08839,1.089444,1.089394,1.089252


In [43]:
backtest_df.rename_axis('DATETIME', inplace=True)
backtest_df.to_csv('../../../backtest_EURUSD_CANDLE.csv')

# Make the trade strategy

## Stratégie avec un ratio perte/gain (Ratio Risk/Reward) de 1:2

perte de 10 pips pour gagner 20 pips
- Stop-loss pips = 0.0010
- Take-profit pips (Stop-loss pips * 2) = 0.0020


In [21]:
dataset.df.tail()

,CLOSE,EMA,RSI,MACD,SIGNAL_LINE,UPPER_BAND,LOWER_BAND,BOLLINGER_PERCENT_B,ATR
2025-03-07 19:00:00,1.08434,1.083537,63.876652,0.001790,0.001785,1.088663,1.077196,0.622998,0.003013
2025-03-07 20:00:00,1.08398,1.083580,60.150943,0.001611,0.001750,1.088624,1.077728,0.573784,0.002918
2025-03-07 21:00:00,1.08425,1.083643,60.888889,0.001474,0.001695,1.088499,1.078403,0.579136,0.002926
2025-03-07 22:00:00,1.08462,1.083736,58.910506,0.001380,0.001632,1.088297,1.079194,0.596069,0.002853
2025-03-07 23:00:00,1.08325,1.083690,51.066961,0.001180,0.001542,1.087942,1.079972,0.411285,0.002878


In [22]:
strategy = TradeStrategy(tk.predict_df, 2)
strategy.run(dataset.df)
strategy.df

,ENTRY_PRICE,PREDICTED_CLOSE,POSITION,STOP_LOSS,TAKE_PROFIT


# Backtesting du model

In [10]:
backtest_data = base_df[['CLOSE']].merge(
    strategy.df, left_index=True, right_index=True
    ).merge(dataset.df[['ATR']], left_index=True, right_index=True)
backtest_data.merge(base_df[['HIGH', 'LOW']], left_index=True, right_index=True, how='inner')

,CLOSE,ENTRY_PRICE,PREDICTED_CLOSE,POSITION,STOP_LOSS,TAKE_PROFIT,ATR,HIGH,LOW
2025-03-06 16:00:00,1.08373,1.08263,1.080028,SELL,1.084908,1.07222,0.002711,1.08411,1.08113


In [11]:
backtester = Backtester(backtest_data)
backtester.simulate_trades()
performance_metrics = backtester.calculate_performance_metrics()

print("Métriques de performance :")
for key, value in performance_metrics.items():
    print(f"{key}: {value}")

Métriques de performance :
total_trades: 1
winning_trades: 0
losing_trades: 1
win_rate: 0.0
net_profit: -0.001100000000000101
average_profit: -0.001100000000000101
sharpe_ratio: nan


In [12]:
# Test de différents multiplicateurs
multipliers = [(1.0, 1.5), (1.5, 2.0), (2.0, 2.5)]

best_metrics = None
best_multipliers = None

for stop_loss_multiplier, take_profit_multiplier in multipliers:
    # Appliquez les multiplicateurs et recalculez les stop-loss et take-profit
    backtest_data['STOP_LOSS'] = backtest_data.apply(
        lambda row: row["PREDICTED_CLOSE"] + (stop_loss_multiplier * row['ATR'])
            if row["POSITION"] == "SELL"
            else row["PREDICTED_CLOSE"] - (stop_loss_multiplier * row['ATR']),
        axis=1
    )
    backtest_data['TAKE_PROFIT'] = backtest_data.apply(
        lambda row: row["PREDICTED_CLOSE"] - (take_profit_multiplier * row['ATR'])
            if row["POSITION"] == "SELL"
            else row["PREDICTED_CLOSE"] + (take_profit_multiplier * row['ATR']),
        axis=1
    )

    # Simule les trades et calcule les métriques
    backtester = Backtester(backtest_data)
    backtester.simulate_trades()
    metrics = backtester.calculate_performance_metrics()

    # Compare les métriques pour trouver les meilleurs paramètres
    if best_metrics is None or metrics['net_profit'] > best_metrics['net_profit']:
        best_metrics = metrics
        best_multipliers = (stop_loss_multiplier, take_profit_multiplier)

print("Meilleurs multiplicateurs :", best_multipliers)
print("Métriques associées :", best_metrics)

Meilleurs multiplicateurs : (1.0, 1.5)
Métriques associées : {'total_trades': 1, 'winning_trades': 0, 'losing_trades': 1, 'win_rate': 0.0, 'net_profit': -0.00010904900021024844, 'average_profit': -0.00010904900021024844, 'sharpe_ratio': nan}
